In [ ]:
# biz_rule1(quarterly) - 파생변수 X (60, 147)
# biz_rule2(quarterly) - 파생변수 O (60, 731)
# biz_rule1(monthly) - 파생변수 X (177, 147)
# biz_rule2(monthly) - 파생변수 O (177, 887)

### 분기별 인과계수 결과 확인, CSV 추출

In [ ]:
# ────────────────────────────────────────────────
# 0. 라이브러리 & 데이터 로딩
# ────────────────────────────────────────────────
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tigramite import data_processing as pp
from tigramite.independence_tests.parcorr import ParCorr
from tigramite.pcmci import PCMCI

# CSV 경로
project_root = r"C:/Users/김현우/PycharmProjects/Causal_Project"
csv_name     = "biz_rule1(monthly).csv"

# ────────────────────────────────────────────────
# 1. 데이터 준비
# ────────────────────────────────────────────────
# (1) 원본 CSV 읽어오기
df = pd.read_csv(os.path.join(project_root, "input", csv_name))

# (2) 'dt' 열을 datetime으로 변환 후 인덱스로 설정
df["dt"] = pd.to_datetime(df["dt"])
df = df.set_index("dt").sort_index()

# (3) 2022년 4분기(2022-12-31)까지의 데이터만 남기기
df = df.loc[: "2023-03-31"]

print("1. 데이터 컷오프 후 shape:", df.shape)

# (4) '_lag'가 붙은 컬럼(과거 시차 컬럼)은 분석에서 제외
df = df[[c for c in df.columns if "_lag" not in c]]

# (5) 식별자용 컬럼('grain_id')이 있으면 삭제
if "grain_id" in df.columns:
    df = df.drop(columns=["grain_id"])

# ────────────────────────────────────────────────
# 2. PCMCI 실행 및 유의한 링크 추출
# ────────────────────────────────────────────────
def get_significant_links_until(dataframe, cutoff_date, tau_max=3, alpha=0.05):
    """
    cutoff_date 이전(포함) 데이터로 PCMCI를 실행한 뒤,
    'pval < alpha'인 링크만 모아서 DataFrame으로 반환합니다.
    반환 컬럼: [Source, Target, lag, pval, val]
    """

    # 2-1. cutoff_date까지 데이터 슬라이싱
    data_cut = dataframe.loc[:cutoff_date]

    # 2-2. 수치형(column dtype이 number인) 컬럼만 추출
    numeric = data_cut.select_dtypes(include=[np.number])

    # 2-3. 표준화(스케일링)
    scaled_df = pd.DataFrame(
        StandardScaler().fit_transform(numeric),
        index=numeric.index,
        columns=numeric.columns
    )

    # 2-4. Tigramite 전용 데이터프레임 생성
    tig_df = pp.DataFrame(
        data      = scaled_df.values.astype(float),
        var_names = list(scaled_df.columns)
    )

    # 2-5. PCMCI 세팅 & 실행
    parcorr = ParCorr(significance="analytic")
    pcmci   = PCMCI(
        dataframe     = tig_df,
        cond_ind_test = parcorr,
        verbosity     = 0   # 0: silent, 1: progress bar, 2: detailed
    )

    res = pcmci.run_pcmci(
        tau_max     = tau_max,
        pc_alpha    = alpha,
        alpha_level = alpha
    )

    # 2-6. 결과(p_matrix, val_matrix)에서 pval < alpha인 링크만 수집
    pmat = res["p_matrix"]   # shape: (변수 수, 변수 수, tau_max+1)
    vmat = res["val_matrix"] # same shape

    rows = []
    for src_idx in range(pmat.shape[0]):
        for trg_idx in range(pmat.shape[1]):
            for lag in range(1, tau_max + 1):
                pval = pmat[src_idx, trg_idx, lag]
                if pval < alpha:
                    rows.append({
                        "Source": tig_df.var_names[src_idx],
                        "Target": tig_df.var_names[trg_idx],
                        "lag":    lag,
                        "pval":   float(pval),
                        "val":    float(vmat[src_idx, trg_idx, lag])
                    })

    # 2-7. DataFrame으로 변환 후 pval 오름차순 정렬
    sig_df = pd.DataFrame(rows).sort_values("pval").reset_index(drop=True)
    return sig_df

# ────────────────────────────────────────────────
# 3. 함수 호출: 2022-Q4까지 유의한 링크 구하기
# ────────────────────────────────────────────────
tbl_22Q4 = get_significant_links_until(df, "2023-03-31",
                                        tau_max=3, alpha=0.05)

print("\n=== 2022-Q4까지 p < 0.05인 모든 인과 링크 개수:", len(tbl_22Q4))

# ────────────────────────────────────────────────
# 4. 결과를 CSV파일로 저장
# ────────────────────────────────────────────────
output_dir = os.path.join(project_root, "output")
os.makedirs(output_dir, exist_ok=True)

csv_path = os.path.join(output_dir, "significant_links_22Q4.csv")

# (1) CSV로 저장
tbl_22Q4.to_csv(csv_path, index=False)

print("CSV 파일 저장 위치:", csv_path)


## 여러 분기별 인과계수 결과 확인, CSV 추출

In [ ]:
# ────────────────────────────────────────────────
# 0. 라이브러리
# ────────────────────────────────────────────────
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tigramite import data_processing as pp
from tigramite.independence_tests.parcorr import ParCorr
from tigramite.pcmci import PCMCI

# CSV 경로
project_root = r"C:/Users/김현우/PycharmProjects/Causal_Project"
csv_name     = "biz_rule1(monthly).csv"

# ────────────────────────────────────────────────
# 1. 데이터 준비
# ────────────────────────────────────────────────
df = pd.read_csv(os.path.join(project_root, "input", csv_name))
df["dt"] = pd.to_datetime(df["dt"])
df = df.set_index("dt").sort_index()
df = df[[c for c in df.columns if "_lag" not in c]]  # lag 컬럼 제거
if "grain_id" in df.columns:
    df = df.drop(columns=["grain_id"])               # 식별자 제거

# ────────────────────────────────────────────────
# 2. PCMCI 실행 함수 정의
# ────────────────────────────────────────────────
def pcmci_table(dataframe, cutoff_date, tau_max=3, alpha=0.05):
    """cutoff_date 이전(포함) 데이터로 PCMCI 실행 후 [source, target, lag, p, val] 테이블 반환"""
    data_cut = dataframe.loc[:cutoff_date]

    numeric = data_cut.select_dtypes(include=[np.number])
    scaled  = pd.DataFrame(
        StandardScaler().fit_transform(numeric),
        index=numeric.index,
        columns=numeric.columns
    )

    tig_df = pp.DataFrame(
        data      = scaled.values.astype(float),
        var_names = list(scaled.columns)
    )

    pcmci = PCMCI(
        dataframe     = tig_df,
        cond_ind_test = ParCorr(significance="analytic"),
        verbosity     = 0
    )
    res = pcmci.run_pcmci(
        tau_max     = tau_max,
        pc_alpha    = alpha,
        alpha_level = alpha
    )

    pmat  = res["p_matrix"]
    vmat  = res["val_matrix"]
    rows  = []
    for src in range(pmat.shape[0]):
        for trg in range(pmat.shape[1]):
            for lag in range(1, tau_max + 1):
                p = pmat[src, trg, lag]
                if p < alpha:
                    rows.append({
                        "Source": tig_df.var_names[src],
                        "Target": tig_df.var_names[trg],
                        "lag":    lag,
                        "pval":   float(p),
                        "val":    float(vmat[src, trg, lag])
                    })
    return pd.DataFrame(rows).sort_values("pval")

# ────────────────────────────────────────────────
# 3. 두 구간 실행 및 저장
# ────────────────────────────────────────────────
tbl_22Q4 = pcmci_table(df, "2022-12-31", tau_max=3, alpha=0.05)
tbl_23Q1 = pcmci_table(df, "2023-03-31", tau_max=3, alpha=0.05)

tbl_22Q4.to_csv("output_pcmci_22Q4.csv", index=False)
tbl_23Q1.to_csv("output_pcmci_23Q1.csv", index=False)

print("PCMCI 결과가 CSV 파일로 저장되었습니다.")


## Significant, Insignificant 평가 (타겟변수 기준, Δval 추가 필요)

In [ ]:
# ────────────────────────────────────────────────
# 0. 라이브러리
# ────────────────────────────────────────────────
import os
import pandas as pd
import numpy as np

# ────────────────────────────────────────────────
# 1. 경로 설정
# ────────────────────────────────────────────────
base_dir   = r"C:/Users/김현우/PycharmProjects/Causal_Project"
output_dir = os.path.join(base_dir, "output")

csv22 = os.path.join(output_dir, "output_pcmci_22Q4.csv")     # 4Q22
csv23 = os.path.join(output_dir, "output_pcmci_23Q1.csv")     # 1Q23
src_data = os.path.join(base_dir, "input", "biz_rule1(monthly).csv")  # 원본 CSV

TARGET = "ASP_monthly"    # ★ 타깃 변수 지정

# ────────────────────────────────────────────────
# 2. 데이터 로드 & Target 필터링
# ────────────────────────────────────────────────
tbl22_all = pd.read_csv(csv22)
tbl23_all = pd.read_csv(csv23)

# Target = ASP_monthly 만 남김
tbl22 = tbl22_all[tbl22_all["Target"] == TARGET]
tbl23 = tbl23_all[tbl23_all["Target"] == TARGET]

# 원본 변수 목록(146개)
df_raw = pd.read_csv(src_data)
vars_full = sorted([c for c in df_raw.columns
                    if "_lag" not in c and c not in ("dt", "grain_id")])

# ────────────────────────────────────────────────
# 3. 유의 Source 집합
# ────────────────────────────────────────────────
sig22 = set(tbl22["Source"])
sig23 = set(tbl23["Source"])

def min_p(df, var):
    return df.loc[df["Source"] == var, "pval"].min() if var in df["Source"].values else np.nan

def max_abs_val(df, var):
    return df.loc[df["Source"] == var, "val"].abs().max() if var in df["Source"].values else np.nan

# ────────────────────────────────────────────────
# 4. 결과 테이블 생성
# ────────────────────────────────────────────────
flag_df = pd.DataFrame({"Variable": vars_full})

flag_df["4Q22_flag"]     = flag_df["Variable"].apply(lambda v: "significant" if v in sig22 else "insignificant")
flag_df["1Q23_flag"]     = flag_df["Variable"].apply(lambda v: "significant" if v in sig23 else "insignificant")
flag_df["4Q22_min_pval"] = flag_df["Variable"].apply(lambda v: min_p(tbl22, v))
flag_df["4Q22_max_val"]  = flag_df["Variable"].apply(lambda v: max_abs_val(tbl22, v))
flag_df["1Q23_min_pval"] = flag_df["Variable"].apply(lambda v: min_p(tbl23, v))
flag_df["1Q23_max_val"]  = flag_df["Variable"].apply(lambda v: max_abs_val(tbl23, v))

# 보기 좋게 포맷팅
flag_df["4Q22_min_pval"] = flag_df["4Q22_min_pval"].apply(lambda x: f"{x:.2e}" if pd.notna(x) else "")
flag_df["1Q23_min_pval"] = flag_df["1Q23_min_pval"].apply(lambda x: f"{x:.2e}" if pd.notna(x) else "")
flag_df["4Q22_max_val"]  = flag_df["4Q22_max_val"].round(4)
flag_df["1Q23_max_val"]  = flag_df["1Q23_max_val"].round(4)

# ────────────────────────────────────────────────
# 5. 엑셀 저장
# ────────────────────────────────────────────────
out_path = os.path.join(output_dir, f"variable_flags_{TARGET}_4Q22_1Q23.xlsx")
flag_df.to_excel(out_path, index=False)
print("엑셀 저장 완료:", out_path)
